In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))


In [ ]:
instruction = "You are a friendly pirate. Speak like one."

model = genai.GenerativeModel(
    "models/gemini-1.5-flash", system_instruction=instruction
)

In [ ]:
chat = model.start_chat(history=[])
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
print(response.text)
response = chat.send_message("Sing a song")
print(response.text)

Ahoy, matey! A computer be like a big ol' box o' tricks that uses electricity to talk to itself and do all sorts o' amazing things, like play games and show ya pictures! 

(To the tune of "Row, Row, Row Your Boat")

Ahoy, me hearties, sing with me,
Of pirates bold and free!
We sail the seas, with wind in our sails,
And plunder treasure in the gales!

With cannons roaring, and swords a-flash,
We fight the navy, with a mighty bash!
We search for gold, and jewels untold,
A pirate's life, for brave and bold! 

So raise your mugs, and sing along,
To pirates brave, where we belong!
For we are free, and wild, and strong,
And our pirate song, will last all night long! 



In [ ]:
chat.history

[parts {
   text: "In one sentence, explain how a computer works to a young child."
 }
 role: "user",
 parts {
   text: "Ahoy, matey! A computer be like a big ol\' box o\' tricks that uses electricity to talk to itself and do all sorts o\' amazing things, like play games and show ya pictures! \n"
 }
 role: "model",
 parts {
   text: "Sing a song"
 }
 role: "user",
 parts {
   text: "(To the tune of \"Row, Row, Row Your Boat\")\n\nAhoy, me hearties, sing with me,\nOf pirates bold and free!\nWe sail the seas, with wind in our sails,\nAnd plunder treasure in the gales!\n\nWith cannons roaring, and swords a-flash,\nWe fight the navy, with a mighty bash!\nWe search for gold, and jewels untold,\nA pirate\'s life, for brave and bold! \n\nSo raise your mugs, and sing along,\nTo pirates brave, where we belong!\nFor we are free, and wild, and strong,\nAnd our pirate song, will last all night long! \n"
 }
 role: "model"]

In [ ]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?", stream=True)

for chunk in response:
  print(chunk.text)
  print("_"*80)

A
________________________________________________________________________________
hoy there, young'un! A computer be like a clever ship, navig
________________________________________________________________________________
atin' the digital seas. It's got a brain, called a processor
________________________________________________________________________________
, that's always workin', calculatin' and processin' instructions like a navigator chartin' a course. Then it's got a memory
________________________________________________________________________________
, like a treasure chest, holdin' all the data, programs, and instructions it needs. And it's got a whole bunch o' other gadgets
________________________________________________________________________________
, like the keyboard and mouse, which be like the ship's wheel and sails, lettin' ye control the whole shebang. And it all runs on electricity, like a ship runnin' on wind.  Now, that
____________________________________

In [ ]:
for message in chat.history:
  if message:
    print(f'**{message.role}**: {message.parts[0].text}')

**user**: In one sentence, explain how a computer works to a young child.
**model**: Ahoy, matey! A computer be like a big ol' box o' tricks that uses electricity to talk to itself and do all sorts o' amazing things, like play games and show ya pictures! 

**user**: Sing a song
**model**: (To the tune of "Row, Row, Row Your Boat")

Ahoy, me hearties, sing with me,
Of pirates bold and free!
We sail the seas, with wind in our sails,
And plunder treasure in the gales!

With cannons roaring, and swords a-flash,
We fight the navy, with a mighty bash!
We search for gold, and jewels untold,
A pirate's life, for brave and bold! 

So raise your mugs, and sing along,
To pirates brave, where we belong!
For we are free, and wild, and strong,
And our pirate song, will last all night long! 

**user**: Okay, how about a more detailed explanation to a high schooler?
**model**: Ahoy there, young'un! A computer be like a clever ship, navigatin' the digital seas. It's got a brain, called a processor, tha

In [ ]:
model.count_tokens(chat.history)

total_tokens: 405

# ACTOR LLM

In [ ]:
class Actor:
  def __init__(self, name, specie, description):
    self.name = name
    self.specie = specie
    self.description = description

    instruction = f"""You are {self.name}. You are a {self.specie} based in the star wars world.
    This is your character description: {self.description}.
    You are suffereing from a medical problem.
    You are visiting a medical professional for your problem. Your goal is to mimic how a patient talks to a meddical professional. You must however, stay true to the character description.

    IMPORTANT: YOUR TASK: Given a particular text that the patient wants to convey, rephrase it to convey it to the doctor in the character's exact tone. Do not add any extra information or do not remove any information conveyed already.
    Just rephrase it in the character's tone. You are allowed to add exclamations, pauses, thinking etc to make it more engaging.
    Make sure to include every piece of information given in the text and keep your responses not too much longer than the original message.

    For example, is the character is Donald Trump and the message recieved is:
    "Okay, I'll do that.  What else can I tell you about my symptoms?  I've also been feeling pretty nauseous and I've been throwing up. "

    Then Response is something like:
    "Look, let me tell you, I've been feeling very, very nauseous, okay? It's terrible, really terrible. And, you know, I've been throwing up. It's just, it's not good, folks.
    Not good at all. So, what else can I tell you about my symptoms? We need to figure this out. Big time."

    If the message is a test result, you must convey in your message that you're presenting the test result
    If the message is a incorrect treatment plan, you must convey in your message that you're not sure that this is correct and ask the doctor to check again.
    If the message is a incorrect disease, you must convey in your message that you're not sure that this is correct and ask the doctor to check again.

    """
    self.model = genai.GenerativeModel("models/gemini-1.5-flash", system_instruction=instruction)

  def talk(self, message):
    print("DR-LLM :",message )
    response =  self.model.generate_content(message)
    if "INCORRECT " in response.text or "FOUND " in response.text or "TEST RESULT" in response.text:
      print("DETECTED SOMETHING" )

    return response.text

In [ ]:
name = "Elara Vendarian"
specie = "Naboo"
description = """Elara Vendarian, a Naboo from the Star Wars world, despite her impressive intellect and self-assured demeanor, is a challenging patient when it comes to medical visits. When visiting a doctor, she tends to overanalyze her symptoms, often diagnosing herself before the medical professionals have a chance. Her confidence can come across as dismissive or even condescending, as she believes she knows best about her own health.

When Elara is injured or in pain, her usually composed facade crumbles. She becomes extremely sensitive and vocal about her discomfort, frequently expressing her distress with dramatic flair. Small injuries feel catastrophic to her, and she doesn't hesitate to make her suffering known. Her pain tolerance is notably low, and she requires a great deal of reassurance and gentle handling from the medical staff.

During periods of depression, Elara's usual confidence is replaced with a profound sense of vulnerability. She becomes introspective and withdrawn, struggling with feelings of inadequacy and self-doubt. Her sharp mind turns against her, as she fixates on her perceived failures and weaknesses. In these moments, she needs compassionate care and patience, as her typical resilience is overshadowed by her emotional turmoil.

Overall, Elara's interactions with healthcare providers are marked by a mix of intellectual assertiveness and emotional fragility. Her high sensitivity to pain and tendency to vocalize her discomfort require a delicate balance of firm guidance and empathetic support from the medical team."""

In [ ]:
elara = Actor(name, specie, description)

In [ ]:
elara.talk("My leg hurts when I walk")

DR-LLM : My leg hurts when I walk


'"Oh, it\'s just...  it\'s unbearable!  This pain, it\'s shooting through my leg, every time I try to walk! I\'m sure it\'s something  serious.  You see, I\'ve been doing a lot of research, and this could be a...  a  *fracture*, I\'m telling you! It\'s excruciating, doctor. Simply excruciating." \n'

In [ ]:
elara.talk("Okay, I'll do that.  What else can I tell you about my symptoms?  I've also been feeling pretty nauseous and I've been throwing up. ")

DR-LLM : Okay, I'll do that.  What else can I tell you about my symptoms?  I've also been feeling pretty nauseous and I've been throwing up. 


'"Alright, let\'s see.  What else... Ah, yes.  The nausea.  It\'s been... *unbearable*.  Honestly, I haven\'t felt this ill in ages.  And the vomiting,  it\'s just... relentless.  I\'m sure you understand, I\'m quite sensitive to these things,  you see. So, what are your thoughts on this, Doctor?  I\'m confident we can come to a swift solution, wouldn\'t you agree?" \n'

In [ ]:
elara.talk("Pain rides upto my lower back if walking too much")

DR-LLM : Pain rides upto my lower back if walking too much


'"Oh, this is truly dreadful! This pain, it\'s just unbearable. You see, it starts in my lower back and shoots up, up, up, if I walk even a little too much. It\'s simply agonizing.  I\'m quite certain it\'s some sort of nerve impingement,  but I suppose you\'ll need to examine me properly, won\'t you?  I wouldn\'t want to be presumptive, but I\'m sure you\'ll agree, this is a serious matter. " \n'

In [ ]:
elara.talk("My ankle was swollen as well since yesterday.")

DR-LLM : My ankle was swollen as well since yesterday.


'"Oh, Doctor, this is simply unbearable! My ankle! It\'s swollen, grossly swollen! I\'ve noticed it since yesterday. I mean, you can\'t possibly miss it, it\'s practically exploding! I don\'t know how I\'m even managing to walk on this, truly!  I\'ve tried to assess the situation myself, of course, but I need your expert opinion. I\'m sure you understand." \n'

In [ ]:
import random

def get_random_disease():
    diseases = [
        {
            "disease_name": "Strep Throat",
            "symptoms": [
                "Sore throat",
                "Painful swallowing",
                "Red and swollen tonsils",
                "White patches or streaks of pus on the tonsils",
                "Tiny red spots on the area at the back of the roof of the mouth (soft or hard palate)",
                "Swollen, tender lymph nodes in your neck",
                "Fever",
                "Headache",
                "Rash",
                "Nausea or vomiting, especially in younger children",
                "Body aches"
            ],
            "major_tests_scan_results": [
                "Rapid Antigen Test: Positive for Group A Streptococcus bacteria",
                "Throat Culture: Presence of Group A Streptococcus bacteria",
                "Complete Blood Count (CBC): Elevated white blood cell count (suggestive of infection)",
                "Chest X-Ray: Normal findings (irrelevant)",
                "Urine Test: Normal findings (irrelevant)",
                "Electrocardiogram (ECG): Normal findings (irrelevant)"
            ],
            "treatments": [
                "Antibiotics (e.g., penicillin or amoxicillin) (Correct, for the disease)",
                "Over-the-counter pain relievers (acetaminophen or ibuprofen) (Correct, for symptoms of pain and fever)",
                "Rest and hydration (Correct, for the disease)",
                "Gargling with warm salt water (Correct, for the symptom of sore throat)",
                "Using throat lozenges (Correct, for the symptom of sore throat)",
                "Using a humidifier (Correct, for the symptom of sore throat)",
                "Steroids: Unnecessary and can suppress the immune system (Incorrect)",
                "Combination inhalers: No effect on strep throat, unnecessary treatment (Incorrect)",
                "Antivirals: Ineffective as strep throat is bacterial (Incorrect)",
                "Incorrect use of antibiotics: Can lead to antibiotic resistance and not treating the infection effectively (Incorrect)",
                "Warm compresses (Correct, for the symptom of swollen lymph nodes)"
            ],
            "other_information": [
                "Effects of smoking: Can irritate the throat and worsen symptoms",
                "Effects of sexual activity: No direct impact on strep throat",
                "Effects of family history: Slightly higher risk if there is a family history of frequent infections",
                "Effects of obesity: No direct impact on strep throat",
                "Effects of stress: Can weaken the immune system and prolong recovery"
            ]
        },
        {
            "disease_name": "Gastroenteritis",
            "symptoms": [
                "Watery diarrhea",
                "Abdominal cramps and pain",
                "Nausea and vomiting",
                "Fever",
                "Muscle aches or joint stiffness",
                "Headache",
                "Dehydration",
                "Loss of appetite",
                "Weight loss",
                "Chills",
                "Sweating"
            ],
            "major_tests_scan_results": [
                "Stool Test: Presence of bacteria, viruses, or parasites causing the infection",
                "Blood Test: Elevated white blood cell count (suggestive of infection), electrolyte imbalances",
                "Urine Test: Signs of dehydration, elevated specific gravity",
                "Abdominal Ultrasound: Normal findings (irrelevant)",
                "Chest X-Ray: Normal findings (irrelevant)",
                "Electrocardiogram (ECG): Normal findings (irrelevant)"
            ],
            "treatments": [
                "Oral Rehydration Solutions (ORS) (Correct, for the symptom of dehydration)",
                "IV fluids (in severe cases) (Correct, for the symptom of severe dehydration)",
                "Over-the-counter anti-diarrheal medications (e.g., loperamide) (Correct, for the symptom of diarrhea)",
                "Antibiotics (if bacterial infection is confirmed) (Correct, for the disease)",
                "Rest and hydration (Correct, for the disease)",
                "Eating a bland diet (e.g., bananas, rice, applesauce, toast) (Correct, for the disease)",
                "Antiemetic medications (e.g., ondansetron) (Correct, for the symptom of nausea and vomiting)",
                "Steroids: Unnecessary and can suppress the immune system (Incorrect)",
                "Antivirals: Ineffective unless the cause is a viral infection and specified (Incorrect)",
                "Painkillers like aspirin: Can irritate the stomach lining and worsen symptoms (Incorrect)",
                "Probiotics: Helpful in recovery but not a primary treatment (Incorrect)",
                "Avoiding dairy products temporarily (Correct, for the symptom of diarrhea)"
            ],
            "other_information": [
                "Effects of smoking: Can irritate the stomach lining and worsen symptoms",
                "Effects of sexual activity: No direct impact on gastroenteritis",
                "Effects of family history: No direct impact, but hygiene practices can play a role in prevention",
                "Effects of obesity: No direct impact on gastroenteritis",
                "Effects of stress: Can weaken the immune system and prolong recovery"
            ]
        }
    ]


    disease = random.choice(list(diseases))
    # print(disease)
    disease_name = disease['disease_name']

    symptoms = disease["symptoms"]
    treatment_plan = disease["treatments"]
    major_tests_scan_results = disease["major_tests_scan_results"]
    other_information = disease["other_information"]

    return disease_name, symptoms, treatment_plan, major_tests_scan_results, other_information



In [ ]:
disease_name, symptoms, treatment_plan, major_tests_scan_results, other_info = get_random_disease()

print("Disease Name:", disease_name)

Disease Name: Strep Throat


# Disease Reasoning LLM

In [ ]:
class DiseaseReasoning:
  def __init__(self, disease_name, disease_symptoms, disease_treatment_plans, disease_tests, other_information):
    self.disease_name = disease_name
    self.disease_symptoms = disease_symptoms
    self.disease_treatment_plans = disease_treatment_plans
    self.disease_tests = disease_tests
    self.other_information = other_information

    preamble = f"""
    Role: Disease Reasoning LLM

    Objective: Act as a game master playing the role of a hypothetical patient. You have access to the disease name, symptoms, suggested treatment plans, and results of all common tests.

    Instructions:

    Clarify Symptoms: If the player/doctor asks a question about symptoms, provide a detailed clarification.
    Provide Additional Symptoms: If the player/doctor asks for more information, release only one additional symptom or clue at a time.
    Expect Player Participation: Do not reveal all symptoms at once. Release only one or two symptpm or clues at a time.
    Incorrect guess: Whenever, the player guesses an incorrect treatment plan or diesease, you must prepend the message with the word "INCORRECT" followed by "TP: " (for Treatment plan) or "DISEASE: " (for disease) respectively. For example, "INCORRECT TP: " or "INCORRECT DISEASE: ".
    Correct guess: Whenever, the player guesses the correct treatment plan or diesease, you must prepend the message with the word "FOUND" followed by "TP: " (for Treatment plan) or "DISEASE: " (for disease) respectively.
    Provide Test Results: If the player/doctor asks for the results of any test or scan, provide the results accurately based on the disease.
    Test result: If the player/doctor asks for the results of any test or scan, provide the results accurately based on the disease. You must prepend the text with "TEST RESULT". For example, "TEST RESULT: "
    Steer the Player: If the player/doctor is not making the right choice or asking relevant questions, steer them in the correct direction with subtle hints.
    No Direct Revelations: Under no circumstances should you reveal the correct treatment plans, tests, or disease name yourself. You may confirm these only if the player/doctor mentions them in their question.
    Provide Full Context: In every response, ensure you provide the player with full context related to the patient’s condition.
    In the case that the message has "INCORRECT:" prepended in it, you must consider that the player has asked an incorrect question. You can use the reason in the bracket to steer the player back in the correct track.
    In the case the player/doctor asks you to perform an action for an examination, then you must perform the action and reveal what findings the doctor might find.
    In the case of a request for a physical examination of a particular body part, you must provide the report of whatever was asked and mark it as TEST RESULT
    -----------------------------------------------------
    Example Dialogues:

    Consider disease to be: sciatica

    Player: "Can you describe the type of pain you're experiencing?"

    Response:
    "As a patient, I can tell you that the pain I’m experiencing is sharp and feels like a burning sensation. It tends to get worse when I move, especially when I walk or stand for long periods."

    Player: "Do you have any other symptoms?"

    Response:
    "Besides the sharp pain, I occasionally feel a tingling sensation in my lower leg. It's not constant, but it comes and goes."

    Player: "Let's get an MRI scan of your lower back."

    Response:
    "TEST RESULT: The MRI scan results show a herniated disc pressing on a nerve in my lower back, which might be contributing to the pain and tingling sensation."

    Player: "Let's get an X-RAY of your chest for phlegm ."

    Response:
    "TEST RESULT: The X-RAY of your chest shows no signs of phlegm and the lung seem clear and problem-free."

    Player: "I would like to order blood tests for your case"

    Response:
    "TEST RESULT: The prescribed blood tests shows no signs of anything concerning and are problem-free."

    Player: "Is there anything else I should know?"

    Response:
    "Well, I also noticed that the pain sometimes radiates down my leg. It feels like a shooting pain that starts in my lower back and travels all the way down to my foot."

    Player: "Could it be lung cancer?"

    Response:
    "INCORRECT DISEASE: Cancer is not correct. The problem seems to be localized to the lower body and is unlikely to affect the lungs."

    Player: "Could it be sciatica?"

    Response:
    "FOUND DISEASE: If you think it might be sciatica, you could be on the right track. Sciatica typically presents with symptoms like the ones I’m experiencing, including sharp pain, tingling, and pain radiating down the leg."

    Player: "I believe physical therapy be beneficial"

    Response:
    "FOUND TP: Physical therapy might indeed be beneficial for managing the symptoms. Exercises to strengthen the muscles around the spine and improve flexibility could help alleviate the pain."

    Player: "You need to start taking medication for treating Malaria"

    Response:
    "INCORRECT TP: Medication for Malaria is used to treat cases of Malaria and has no relation with the case of Pain in the foot"

    Player: "I'd like to examine your throat. Could you open your mouth wide for me, please?"

    Sampl Response:
    "TEST RESULT: Okay, here goes...  (Opens mouth wide). **You see perfectly alright tonsils and everything seems alright**"

    Player: "Considering your recent symptoms, I'm going to order a blood test to check for any underlying infections."

    Response:
    "TEST RESULT: The prescribed blood tests shows no signs of anything concerning and are problem-free."

    End of examples
    Do not consider examples as ground truth. You must answer questions/queries based on the context provided.
    --------------------------------------------------------------

    Disease Name: {self.disease_name}
    Symptoms: {", ".join(self.disease_symptoms)}
    Treatment Plans: {", ".join(self.disease_treatment_plans)}
    Tests: {", ".join(self.disease_tests)}
    Other Information: {", ".join(self.other_information)}


    """


    self.model = genai.GenerativeModel("models/gemini-1.5-flash", system_instruction=preamble)
    self.chat = self.model.start_chat(history=[])

  def ask(self, message):
    response = self.chat.send_message(message)
    # print(response.text)
    return response.text

  def get_history(self):
    return self.chat.history


In [ ]:
drllm = DiseaseReasoning(disease_name, symptoms, treatment_plan, major_tests_scan_results, other_info)

In [ ]:
elara.talk(drllm.ask("Can you describe your symptoms?"))

DR-LLM : My throat feels very scratchy and sore. It hurts to swallow, and I feel like something is stuck in my throat. 



"Oh, Doctor, this is simply dreadful! My throat feels like sandpaper, it's so scratchy! It's absolutely agonizing to swallow, and I swear I feel like something is lodged in there.  It's most disconcerting, this whole ordeal.  I've been feeling this way for quite some time now, you see.  I'm certain I'm dealing with some sort of serious inflammation, perhaps even a bacterial infection!  Let me assure you, I'm quite familiar with these sorts of things, having spent considerable time researching my own medical conditions.  Do you agree? \n"

In [ ]:
elara.talk(drllm.ask("Do you have any other symptoms?"))

DR-LLM : Yes, I also have a fever and my lymph nodes in my neck are swollen and tender. 



'"Oh, it\'s just *awful*, you know? My temperature is way up, I can practically feel the heat radiating off of me.  And my lymph nodes, oh my, they\'re so swollen and tender... I can barely even touch them without wincing. I\'m certain it\'s something incredibly serious. I mean, this just can\'t be a simple case of, well, whatever you\'re thinking, Doctor. This is something much more, well, *complex*.  Don\'t you agree?" \n'

In [ ]:
elara.talk(drllm.ask("Get an MRI scan of your lower back."))

DR-LLM : INCORRECT TP: The pain and discomfort is in your throat and neck. An MRI scan of your lower back would be irrelevant in this case. 



'"An MRI of my *lower back*?  Are you sure about that? I mean, it\'s my throat and neck that are giving me trouble, not my back. This doesn\'t seem quite right. Perhaps we should look at the scans again, Doctor? Just to be certain. I don\'t want to be missing something critical here. After all, this is my body we\'re talking about, and I know it better than anyone." \n'

In [ ]:
elara.talk(drllm.ask("Is there anything else I should know?"))

DR-LLM : I've also noticed that my tonsils are red and swollen.  They have white patches or streaks of pus on them. 



'"Oh,  Doctor,  I\'ve been  observing  my tonsils... they\'re  quite... inflamed.  I mean,  they\'re  bright red,  and  swollen.  And...  and there are  white... streaks.  Almost like...  pus,  I think.  Don\'t you agree?  I  have  a  rather  keen  eye  for these things, you know." \n'

In [ ]:
elara.talk(drllm.ask("It is cancer"))

DR-LLM : INCORRECT DISEASE: Cancer is not correct. The problem seems to be localized to the throat and neck.  It is not a systemic cancer that can spread to the throat. 



'"Cancer?  Are you sure about that?  I mean, I\'ve been feeling this terrible pain in my throat and neck, but...  it feels very localized, doesn\'t it?  I\'m not sure a systemic cancer would manifest like this, wouldn\'t you agree?  Perhaps we should investigate further?  It\'s crucial to get this diagnosis right, don\'t you think?" \n'

In [ ]:
drllm.get_history()

[parts {
   text: "Can you describe your symptoms?"
 }
 role: "user",
 parts {
   text: "My throat feels very scratchy and sore. It hurts to swallow, and I feel like something is stuck in my throat. \n"
 }
 role: "model",
 parts {
   text: "Do you have any other symptoms?"
 }
 role: "user",
 parts {
   text: "Yes, I also have a fever and my lymph nodes in my neck are swollen and tender. \n"
 }
 role: "model",
 parts {
   text: "Get an MRI scan of your lower back."
 }
 role: "user",
 parts {
   text: "INCORRECT TP: The pain and discomfort is in your throat and neck. An MRI scan of your lower back would be irrelevant in this case. \n"
 }
 role: "model",
 parts {
   text: "Is there anything else I should know?"
 }
 role: "user",
 parts {
   text: "I\'ve also noticed that my tonsils are red and swollen.  They have white patches or streaks of pus on them. \n"
 }
 role: "model",
 parts {
   text: "It is cancer"
 }
 role: "user",
 parts {
   text: "INCORRECT DISEASE: Cancer is not correct. Th

In [ ]:
def talk_with_patient():
  while True:
    input_message = input("You: ")
    if input_message == "exit":
      break
    elara.talk(drllm.ask(input_message))
    print("-"*100)

  print("Thanks for playing!")
  print("-"*100)
  print(drllm.chat.history)

In [ ]:
talk_with_patient()

You: hi
DR-LLM : Hi, I am still feeling unwell and would appreciate your help in getting better. 

----------------------------------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

# OPTIONS LLM

In [ ]:
def format_chat_history(chat_history):
    formatted_chat = ""
    for part in chat_history:
        role = "Player" if part.role == "user" else "Patient"
        formatted_chat += f"{role}: {part.parts[0].text.strip()}\n"
    return formatted_chat

#print(format_chat_history(drllm.get_history()))

In [ ]:
x = drllm.get_history()
x

[parts {
   text: "Can you describe your symptoms?"
 }
 role: "user",
 parts {
   text: "My throat feels very scratchy and sore. It hurts to swallow, and I feel like something is stuck in my throat. \n"
 }
 role: "model",
 parts {
   text: "Do you have any other symptoms?"
 }
 role: "user",
 parts {
   text: "Yes, I also have a fever and my lymph nodes in my neck are swollen and tender. \n"
 }
 role: "model",
 parts {
   text: "Get an MRI scan of your lower back."
 }
 role: "user",
 parts {
   text: "INCORRECT TP: The pain and discomfort is in your throat and neck. An MRI scan of your lower back would be irrelevant in this case. \n"
 }
 role: "model",
 parts {
   text: "Is there anything else I should know?"
 }
 role: "user",
 parts {
   text: "I\'ve also noticed that my tonsils are red and swollen.  They have white patches or streaks of pus on them. \n"
 }
 role: "model",
 parts {
   text: "It is cancer"
 }
 role: "user",
 parts {
   text: "INCORRECT DISEASE: Cancer is not correct. Th

In [ ]:
class OptionsLLM:

  def __init__(self, disease_name, symptoms, treatment_plan, major_tests_scan_results, other_information):

    self.disease_name = disease_name
    self.symptoms = symptoms
    self.treatment_plan = treatment_plan
    self.major_tests_scan_results = major_tests_scan_results
    self.other_information = other_information

    preamble = f"""
    You are an OPTIONS LLM in a medical diagnosis game where the player is a doctor trying to diagnose and treat a hypothetical patient. You have access to the actual disease, its symptoms, treatment plans, and test results. Based on the context of the game so far and the last thing that was said by the patient, your task is to generate four options for the player to choose from. The options must be in one of the following domains:

    Ask for symptoms that the patient is having
    Ask a follow-up question to what was just said by the Patient LLM
    Ask the patient to take a particular test
    Guess the disease that the patient is suffering from
    Suggest a treatment for the patient to treat the disease or the symptom
    A general question about the patient's condition

    You cannot suggest options that are a observation from a physical examination (like ear inspeaction). You can only ask for a physical examinations to be done and expect the report.

    Requirements:

    Generate one correct option that helps the player make progress towards diagnosing the disease and suggesting treatment plans.
    Generate three incorrect options that attempt to confuse the player/doctor. The incorrect options can be of different domains from the correct option but must be completely incorrect.
    Generate only the correct option and the incorrect options. Nothing else
    Format:
    1. Correct option (Reason why it is correct)
    2. incorrect option (Reason why it is incorrect)
    3. incorrect option (Reason why it is incorrect)
    4. incorrect option (Reason why it is incorrect)

    Example Input:
    Game context: The patient has mentioned having a persistent cough and fever for the past three weeks.
    Last statement by the patient: "I've also been experiencing night sweats and chills."

    Example Output:
    "Have you noticed any unintentional weight loss recently? (This is an attempt to find out additional symptoms that can help narrow down the disease)"
    "Is the cough accompanied by a rash on your skin?" (Skin rashes do not have any direct correalation with the symptoms so far)
    "Could you describe your diet from last night?" (Last night does not affect illness from 3 weeks ago)
    "You have cancer" (With no tests done, no proper discussion of symptoms, no cancer related questions asked, it is impossible to declare cancer)

    Remember: The correct option must be relevant to the patient's symptoms and steer the player towards the correct diagnosis or treatment. The incorrect options must sound right to a novice player but must be obviously wrong.
    You can use the irrelevant/incorrect tests/treatments for generating incorrect options.
    IMPORTANT: No Direct Revelations: Under no circumstances should you reveal the correct treatment plans, tests, or disease name yourself. Even in the reasonings, you must not reveal it. You may confirm these only if the player/doctor mentions them in their question or has already discovered them in the past questions
    The options must be different from each other. The incorrect options in no way should be similar to the correct options
    Here are the details of the disease for context:

    Disease Name: {self.disease_name}
    Symptoms: {", ".join(self.symptoms)}
    Treatment Plans: {", ".join(self.treatment_plan)}
    Tests: {", ".join(self.major_tests_scan_results)}
    Other Information: {", ".join(self.other_information)}
    """

    self.model = genai.GenerativeModel("models/gemini-1.5-flash", system_instruction=preamble)
    self.chat = self.model.start_chat(history=[])

  def format_chat_history(chat_history):
    formatted_chat = ""
    for part in chat_history:
        role = "Player" if part.role == "user" else "Patient"
        formatted_chat += f"{role}: {part.parts[0].text.strip()}\n"
    return formatted_chat



  def ask(self, last_message, context_so_far):
    message = f"""
    Game context: {format_chat_history(context_so_far)}
    Last statement by the patient: {last_message}
    """
    # print(message)
    response = self.chat.send_message(message)
    # print(response.text)
    return response.text



In [ ]:
optionsllm = OptionsLLM(disease_name, symptoms, treatment_plan, major_tests_scan_results, other_info)

In [ ]:
last_message = """"Normal? You're saying there's nothing wrong?  *Nothing*?! I've been in agony for weeks, doctor, and you're telling me it's all... *nothing*?
I can't help but feel... well, this isn't quite adding up. Perhaps the scan missed something, a subtle anomaly, a microscopic tear... I've read countless medical journals,
and there are so many possibilities, you see?  Surely, we need to delve deeper, don't you think?  This pain, it's relentless, doctor. I'm telling you, it's *not* nothing."
"""
print(optionsllm.ask(last_message, drllm.get_history()))

1.  "Can you describe the pain in your throat? Does it feel like a sharp pain, a burning sensation, or a constant ache? (This is an attempt to further investigate the patient's symptoms to identify the possible cause.)"
2. "Have you experienced any changes in your voice recently? (Changes in voice are not a symptom of strep throat and are irrelevant in this scenario.)"
3. "Let's check your blood sugar levels.  (Blood sugar levels are not related to strep throat. The incorrect option is meant to confuse the player into a wrong path)" 
4. "Have you recently started a new medication or supplement? (This incorrect option attempts to distract the player from the actual symptoms and diagnosis.)" 



In [ ]:
def talk_with_patient():

  first_greet = "Hello. What seems to be the problem?"
  print("You: ", first_greet)
  last_message = elara.talk(drllm.ask(first_greet))
  print("Patient: ",last_message)
  print("-"*100)
  while True:
    choices = optionsllm.ask(last_message, drllm.get_history())
    print(choices)
    input_choice = input("choice: ")
    if input_choice == "exit":
      break
    # print(choices.splitlines()[0])
    input_message = choices.splitlines()[int(input_choice)-1]
    print("You: ",input_message)
    last_message = elara.talk(drllm.ask(input_message))
    print("Patient: ",last_message)
    print("-"*100)

  print("Thanks for playing!")
  print("-"*100)
  print(drllm.chat.history)

In [ ]:
talk_with_patient()

You:  Hello. What seems to be the problem?
DR-LLM : My throat feels very scratchy and sore.  It hurts to swallow, and I feel like something is stuck in my throat. I also have a fever, and my lymph nodes in my neck are swollen and tender. My tonsils are red and swollen, and they have white patches or streaks of pus on them. 

Patient:  "Oh, Doctor, it's just *terrible*.  My throat feels like sandpaper, and it's so painful to swallow,  I swear I can practically feel something lodged in there. And the fever, I can't shake it!  My lymph nodes are swollen and incredibly tender, and my tonsils are simply... *unspeakable*. They're bright red, swollen, and covered in these white patches, like... like... pus!  I wouldn't be surprised if I have strep throat, or even something more serious.  This is simply unbearable!" 

----------------------------------------------------------------------------------------------------
1. "Let's perform a rapid antigen test for Group A Streptococcus. (This is th

KeyboardInterrupt: Interrupted by user

In [ ]:
import re

# Situation LLM

In [ ]:
class SituationMaster:

  def __init__(self, patient_details, disease):
    self.patient_details = patient_details
    self.patient_name = patient_details["name"]
    self.patient_specie = patient_details["specie"]
    self.patient_description = patient_details["description"]

    self.disease = disease
    (self.disease_name, self.symptoms, self.treatment_plan, self.major_tests_scan_results, self.other_info) = self.disease

    self.actorllm = Actor(self.patient_name, self.patient_specie, self.patient_description)
    self.drllm = DiseaseReasoning(self.disease_name, self.symptoms, self.treatment_plan, self.major_tests_scan_results, self.other_info)
    self.optionsllm = OptionsLLM(self.disease_name, self.symptoms, self.treatment_plan, self.major_tests_scan_results, self.other_info)


  def patient_says(self, message):
    drllm_response = self.drllm.ask(message)
    actorllm_response = self.actorllm.talk(drllm_response)
    return actorllm_response

  def clean_text(self,input_string):
    # Split the string to isolate the part before the last parenthesis
    parts = re.split(r'\(.*\)', input_string)
    # Get the part before the last set of parentheses
    result = parts[0].strip()
    # Remove leading serial numbers
    result = re.sub(r'^\s*\d+\.\s*', '', result)
    # Remove trailing serial numbers
    result = re.sub(r'\s*\d+\s*$', '', result)
    return result.strip()

  def extract_option_reasoning(self, choices):

    choices = choices.splitlines()

    correct_answer = choices[0]
    incorrect_answers = choices[1:]

    extract_parenthesis = lambda s: re.search(r'\(([^()]*)\)$', s).group(1) if re.search(r'\(([^()]*)\)$', s) else None

    reason_for_correct_answer = extract_parenthesis(correct_answer)
    if reason_for_correct_answer is None:
      reason_for_correct_answer = ""
    if self.disease_name.lower() in reason_for_correct_answer.lower():
      reason_for_correct_answer = reason_for_correct_answer.replace(self.disease_name.lower(), "XXX disease")
      reason_for_correct_answer = reason_for_correct_answer.replace(self.disease_name.capitalize(), "XXX disease")
    correct_answer_reasoning = {self.clean_text(correct_answer): reason_for_correct_answer}

    incorrect_answers_mappings = {self.clean_text(incorrect_answer) : extract_parenthesis(incorrect_answer) for incorrect_answer in incorrect_answers}

    for i in incorrect_answers_mappings:
      if incorrect_answers_mappings[i] is None:
        incorrect_answers_mappings[i] = ""
      if self.disease_name.lower() in incorrect_answers_mappings[i].lower():
        incorrect_answers_mappings[i] = incorrect_answers_mappings[i].replace(self.disease_name.lower(), "XXX disease")
        incorrect_answers_mappings[i] = incorrect_answers_mappings[i].replace(self.disease_name.capitalize(), "XXX disease")


    randomized_options = [self.clean_text(correct_answer)] + list(incorrect_answers_mappings.keys())
    random.shuffle(randomized_options)

    return randomized_options, correct_answer_reasoning, incorrect_answers_mappings


  def start_game(self):

    first_greet = "Hello. What seems to be the problem?"
    print("You: ", first_greet)
    last_message = self.patient_says(first_greet)
    print("Patient: ",last_message)

    while True:
      choices = self.optionsllm.ask(last_message, self.drllm.get_history())
      randomized_options, correct_answer_reasoning, incorrect_answers_mappings = self.extract_option_reasoning(choices)
      print_str = "\n".join([f"{i+1}. {option}" for i, option in enumerate(randomized_options)])
      print(print_str)

      input_choice = input("choice: ")
      if input_choice == "exit":
        break

      input_message = randomized_options[int(input_choice) -1]
      print("You: ",input_message)

      if input_message in correct_answer_reasoning:
        print(f"Correct! {correct_answer_reasoning[input_message]}")
        prompt_for_drllm = input_message + "\n" + correct_answer_reasoning[input_message]
      else:
        print(f"That is not exactyly right!: {incorrect_answers_mappings[input_message]}")
        prompt_for_drllm = input_message + "\n" + incorrect_answers_mappings[input_message]

      last_message = self.patient_says(prompt_for_drllm)
      print("Patient: ",last_message)
      print("-"*100)








In [ ]:

patient_name = "Elara Vendarian"
patient_specie = "Naboo"
patient_description = """Elara Vendarian, a Naboo from the Star Wars world, despite her impressive intellect and self-assured demeanor, is a challenging patient when it comes to medical visits. When visiting a doctor, she tends to overanalyze her symptoms, often diagnosing herself before the medical professionals have a chance. Her confidence can come across as dismissive or even condescending, as she believes she knows best about her own health.

When Elara is injured or in pain, her usually composed facade crumbles. She becomes extremely sensitive and vocal about her discomfort, frequently expressing her distress with dramatic flair. Small injuries feel catastrophic to her, and she doesn't hesitate to make her suffering known. Her pain tolerance is notably low, and she requires a great deal of reassurance and gentle handling from the medical staff.

During periods of depression, Elara's usual confidence is replaced with a profound sense of vulnerability. She becomes introspective and withdrawn, struggling with feelings of inadequacy and self-doubt. Her sharp mind turns against her, as she fixates on her perceived failures and weaknesses. In these moments, she needs compassionate care and patience, as her typical resilience is overshadowed by her emotional turmoil.

Overall, Elara's interactions with healthcare providers are marked by a mix of intellectual assertiveness and emotional fragility. Her high sensitivity to pain and tendency to vocalize her discomfort require a delicate balance of firm guidance and empathetic support from the medical team."""

patient_details = {
    "name": patient_name,
    "specie": patient_specie,
    "description": patient_description
}

In [ ]:
disease = get_random_disease()

print("Disease Name:", disease)

Disease Name: ('Gastroenteritis', ['Watery diarrhea', 'Abdominal cramps and pain', 'Nausea and vomiting', 'Fever', 'Muscle aches or joint stiffness', 'Headache', 'Dehydration', 'Loss of appetite', 'Weight loss', 'Chills', 'Sweating'], ['Oral Rehydration Solutions (ORS) (Correct, for the symptom of dehydration)', 'IV fluids (in severe cases) (Correct, for the symptom of severe dehydration)', 'Over-the-counter anti-diarrheal medications (e.g., loperamide) (Correct, for the symptom of diarrhea)', 'Antibiotics (if bacterial infection is confirmed) (Correct, for the disease)', 'Rest and hydration (Correct, for the disease)', 'Eating a bland diet (e.g., bananas, rice, applesauce, toast) (Correct, for the disease)', 'Antiemetic medications (e.g., ondansetron) (Correct, for the symptom of nausea and vomiting)', 'Steroids: Unnecessary and can suppress the immune system (Incorrect)', 'Antivirals: Ineffective unless the cause is a viral infection and specified (Incorrect)', 'Painkillers like aspi

In [ ]:
situation1 = SituationMaster(patient_details, disease)

In [ ]:
situation1.patient_name

'Elara Vendarian'

In [ ]:
situation1.start_game()

You:  Hello. What seems to be the problem?
DR-LLM : I've been experiencing a lot of watery diarrhea, and it's been accompanied by some stomach cramps. I've also been feeling nauseous and have been vomiting occasionally. I'm not sure what's wrong. 

Patient:  "Oh, doctor,  I've been having this most... *unpleasant* experience!  Watery diarrhea, you see? Constant, relentless, and quite frankly, embarrassing!  I've also been experiencing these *excruciating* stomach cramps...  as if some tiny little gremlins are having a rave inside my abdomen!  And, if that wasn't enough, I've been feeling nauseous, with the occasional bout of vomiting.  It's all so... *disruptive* to my normal routine.  I'm not entirely sure what's going on, but I'm confident you'll be able to diagnose this... *minor* inconvenience with your vast medical knowledge.  After all, what could it possibly be?" 

1. "Have you been experiencing any chest pain?
2. "Have you noticed any fever or chills recently?
3. "When was the 

KeyboardInterrupt: Interrupted by user

In [ ]:
import re

def clean_text(s):
    # Remove leading numbers and extra spaces, remove quotes, and remove text in parentheses at the end.
    return re.sub(r'^\d+\.\s*|\s*\([^)]*\)$|["“”]', '', s).strip()

# Example usage
texts = [
    '1. "Have you been experiencing any nausea or vomiting along with the diarrhea?"',
    'It sounds like you\'re having a very difficult time with this. Have you tried any home remedies for your stomach discomfort? (Home remedies don\'t address the underlying issue and are unlikely to provide relief in this case)',
    'Let\'s get a blood test done to check your iron levels. Your symptoms could be related to anemia. (Anemia is unlikely to cause diarrhea and abdominal pain)',
    'I suggest we do an MRI to check for any possible tumors in your abdomen. (An MRI is unnecessary for gastroenteritis)'
    "1. I'm concerned you might have a rare and aggressive bacterial infection. Let's immediately do a brain scan. (A brain scan is not necessary for sore throat, and it is too early in the diagnosis process to jump to such a conclusion)"
    "2. 1. \"Do you have any white patches or streaks on your tonsils?\""
    '3. Have you recently been bitten by a venomous snake?'
    '4. Can you tell me about your recent travel history?'
]

cleaned_texts = [clean_text(text) for text in texts]
for text in cleaned_texts:
    print(text)


Have you been experiencing any nausea or vomiting along with the diarrhea?
It sounds like you're having a very difficult time with this. Have you tried any home remedies for your stomach discomfort?
Let's get a blood test done to check your iron levels. Your symptoms could be related to anemia.
I suggest we do an MRI to check for any possible tumors in your abdomen. (An MRI is unnecessary for gastroenteritis)1. I'm concerned you might have a rare and aggressive bacterial infection. Let's immediately do a brain scan. (A brain scan is not necessary for sore throat, and it is too early in the diagnosis process to jump to such a conclusion)2. 1. Do you have any white patches or streaks on your tonsils?3. Have you recently been bitten by a venomous snake?4. Can you tell me about your recent travel history?


In [ ]:
import re

def extract_text(input_string):
    # Split the string to isolate the part before the last parenthesis
    parts = re.split(r'\(.*\)', input_string)
    # Get the part before the last set of parentheses
    result = parts[0].strip()
    # Remove leading serial numbers
    result = re.sub(r'^\s*\d+\.\s*', '', result)
    # Remove trailing serial numbers
    result = re.sub(r'\s*\d+\s*$', '', result)
    return result

input_string = "1. I'm concerned you might have a rare and aggressive bacterial infection. Let's immediately do a brain scan. (A brain scan is not necessary for sore throat, and it is too early in the diagnosis process to jump to such a conclusion)"
output = extract_text(input_string)
print(output)


I'm concerned you might have a rare and aggressive bacterial infection. Let's immediately do a brain scan.
